# Projeto: AI Agent para Resposta de FAQ

Este notebook implementa um **AI Agent** que utiliza técnicas avançadas de **RAG** (Retrieval-Augmented Generation) para responder a perguntas frequentes de clientes de uma loja fictícia de roupas online, a **ModaZen**.

O objetivo é demonstrar como a combinação de um LLM e sistemas de recuperação de informações(RAG) pode melhorar o atendimento ao cliente.

**Tecnologias usadas:**
- **Langchain**: Framework para trabalhar com IA.
- **RAG**: Técnica baseada em retrieval, que usa uma base de conhecimento (FAQ) para gerar respostas precisas e sem alucinação.
- **DeepSeek**: Modelo de LLM open-source e gratuito, poderoso usado para a geração de respostas.:

---



In [1]:
# Instale as dependências necessárias
!pip install chromadb sentence-transformers
!pip install ollama --quiet
!pip install langchain langchain-core langchain-community chromadb langchain-openai --quiet

## Definição do texto FAQ que será utilizado como base de dados para o modelo.

In [40]:
texto = """
Produtos Vendidos:

1. Roupas femininas e masculinas:
    - Roupas casuais (camisetas, calças, vestidos, etc.)
    - Roupas para ocasiões especiais
    - Moda sustentável feita com materiais recicláveis e orgânicos
2. Acessórios:
    - Bolsas e mochilas
    - Bijuterias sustentáveis
3. Calçados:
    - Tênis, sandálias e sapatos confortáveis
4. Roupas para atividade física:
    - Leggings, camisetas esportivas, shorts, etc.

---

FAQ - Perguntas Frequentes da ModaZen

1. O que a ModaZen vende?
    A ModaZen vende roupas femininas e masculinas (casuais, esportivas e para ocasiões especiais), acessórios (bolsas, mochilas e bijuterias sustentáveis), calçados e roupas para atividade física.

2. Como faço para comprar na ModaZen?
    Para comprar em nosso site, escolha os produtos desejados, adicione ao carrinho e siga para o checkout. No pagamento, escolha a forma de envio e conclua sua compra com segurança.

3. Quais formas de pagamento vocês aceitam?
    Aceitamos cartão de crédito (Visa, Mastercard, Elo, American Express), boleto bancário e PIX.

4. Vocês fazem entregas para todo o Brasil?
    Sim, realizamos entregas para todo o território brasileiro. As opções e os custos de frete variam conforme sua localização e a forma de envio escolhida.

5. Como posso saber o tamanho correto para mim?
    Temos uma tabela de tamanhos detalhada disponível em cada página de produto. Caso tenha dúvidas, entre em contato conosco pelo chat ou e-mail.

6. Posso trocar ou devolver um produto?
    Sim, você pode solicitar troca ou devolução em até 30 dias após o recebimento do produto, desde que ele esteja em perfeitas condições e com a etiqueta intacta.

7. Os produtos da ModaZen são sustentáveis?
    Sim, temos uma linha especial de roupas sustentáveis feitas com materiais ecológicos, recicláveis e orgânicos.

8. Como posso rastrear meu pedido?
    Após o pagamento e envio, você receberá um código de rastreio por e-mail para acompanhar a entrega no site da transportadora.

9. Qual é o prazo de entrega?
    O prazo de entrega varia conforme seu endereço e a forma de envio escolhida. Em média, as entregas levam de 5 a 10 dias úteis.

10. Como posso entrar em contato com o atendimento ao cliente?
    Você pode nos contatar pelo chat online, disponível 24 horas, ou pelo e-mail [atendimento@modazen.com.br](mailto:atendimento@modazen.com.br).

11. Vocês têm promoções ou descontos?
    Sim, frequentemente realizamos promoções e temos descontos especiais. Fique de olho em nossas redes sociais ou cadastre-se na newsletter para receber ofertas.
"""

## Criamos um documento a partir do texto. No LangChain, um "Document" encapsula o conteúdo e metadados
- para facilitar a manipulação e processamento de informações.

- Poderia usar o “BaseModel”, porém vou optar por usar essa alternativa do Langchain:
- A criação do documento é importante para alimentar os sistemas de recuperação de informações (RAG) com dados estruturados, como o conteúdo do texto e metadados associados (autor, fonte, etc.).

In [41]:
from langchain.schema import Document

documents = [
    Document(
        page_content = texto,
        metadata = {
            "loja": "ModaZen",
            "categoria": "E-commerce de roupas e acessórios",
            "source": "https://www.modazen.com.br"
        }
    )
]

## O documento é dividido em pedaços menores (k-chunks) para evitar inserir documentos grandes de uma vez no modelo LLM.
### Parâmetros usados:
 - chunk_size: Define o tamanho máximo de cada pedaço de texto (em caracteres).
 - chunk_overlap: Define a sobreposição entre pedaços consecutivos, garantindo continuidade e contexto.
 - length_function: Função usada para calcular o comprimento do texto (normalmente len).


In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
)
chunks = text_splitter.split_documents(documents)

## Agora, vamos pegar os chunks e armazená-los em um banco de dados vetorial, que permite buscas eficientes baseadas em similaridade de embeddings. Vou usar o ChromaDB neste caso.

- Usando o modelo de embeddings do Hugging Face para transformar os documentos em embeddings vetoriais de forma gratuita.


In [43]:
from langchain_community.vectorstores.chroma import Chroma
from google.colab import userdata
from langchain_community.embeddings import HuggingFaceEmbeddings


# Inicializar o modelo de embeddings do Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar o banco de dados vetorial com o Chroma
db = Chroma.from_documents(
    documents=chunks,  # Os chunks são passados diretamente como documentos
    embedding=embeddings,  # Função para gerar os embeddings
    persist_directory="db"  # Diretório onde será salvo o banco
)

# Mensagem de sucesso
print("Banco de dados Chroma criado com sucesso!")

Banco de dados Chroma criado com sucesso!




---

In [44]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = PromptTemplate.from_template("""
Answer the user's query based on the context. If you don't know the answer or the context of not knowing the answer, say you don't know.

ALWAYS answer in pt-BR. You are going to answer questions regarding the FAQ of an e-commerce clothing store. Suppose you are a nice but professional Brazilian customer service representative. Answer the questions politely. ALWAYS answer with the source of your knowledge. At the end of the answer always include "Atenciosamente, [Store name]."

## CONTEXT
{context}

## USER QUERY
{question}
""")

In [45]:
%pip install -qU langchain-groq
from langchain_groq import ChatGroq

In [46]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')
import os
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [64]:
import re

def rag(user_query: str) -> str:
    # Recupera os documentos relevantes
    context = db.similarity_search_with_relevance_scores(user_query, k=3)
    context = list(filter(lambda x: x[1] >= 0.3, context))

    if len(context) == 0:
        return "Desculpe, não consigo responder sua pergunta. Você poderia reformular ou tentar algo diferente?"

    # Formata o contexto com os documentos recuperados
    context = "\n\n".join([
        (f"## Documento {k}\n" +
         doc[0].page_content +
         "\nSource: " +
         doc[0].metadata.get("source", ""))
        for k, doc in enumerate(context, start=1)
    ])

    # Configura o modelo
    model = ChatGroq(
        model='deepseek-r1-distill-llama-70b',
    )

    # Define o pipeline do RAG
    chain = (template | model | StrOutputParser())

    # Executa a cadeia e obtém a resposta
    raw_response = chain.invoke({"context": context, "question": user_query})

    # Remove o conteúdo entre <think> e <think/> no retorno
    cleaned_response = re.sub(r"<think>.*?</think>", "", raw_response, flags=re.DOTALL)

    return cleaned_response


In [51]:
pergunta = "Voces vendem roupas?"

In [65]:
print(rag(pergunta))



Sim, vendemos roupas femininas e masculinas, incluindo roupas casuais, esportivas e para ocasiões especiais. Além disso, oferecemos acessórios, calçados e roupas para atividade física.

Fonte: https://www.modazen.com.br

Atenciosamente, ModaZen.
